In [1]:
import pandas as pd
import numpy as np

In [2]:
# users = pd.read_csv('users.csv', on_bad_lines='skip')
users = pd.read_csv("users.csv", encoding="latin1")

In [3]:
positions = pd.read_csv('positions.csv')

In [4]:
edu = pd.read_csv('edu.csv')

In [5]:
print("--- Data Shape ---")
print(f"Users: {users.shape}")
print(f"Positions: {positions.shape}")
print(f"Education: {edu.shape}\n")

--- Data Shape ---
Users: (273074, 29)
Positions: (39066, 35)
Education: (11401, 25)



In [6]:
# --- Step 1: Standardize Column Names ---
# Best practice: Convert all column names to lowercase to avoid case sensitivity issues.
print("Step 1: Standardizing column names to lowercase...")
users.columns = users.columns.str.lower()
positions.columns = positions.columns.str.lower()
edu.columns = edu.columns.str.lower()
print("Column names standardized.\n")

Step 1: Standardizing column names to lowercase...
Column names standardized.



In [7]:
users['updated_dt'] = pd.to_datetime(users['updated_dt'], errors='coerce')
users_sorted = users.sort_values(by=['user_id', 'updated_dt'], ascending=[True, False])
users_deduplicated = users_sorted.drop_duplicates(subset='user_id', keep='first').copy()
print(f"\n After deduplication, the shape of the users table: {users_deduplicated.shape}")
print(f"After deduplication, the number of unique IDs in the users table: {users_deduplicated['user_id'].nunique()}")
print("✅ The 'users' table has only one record for each user_id.\n")


 After deduplication, the shape of the users table: (4757, 29)
After deduplication, the number of unique IDs in the users table: 4757
✅ The 'users' table has only one record for each user_id.



In [8]:
# --- 步骤 1: 筛选用户并选择特征 ---

# 1a. 筛选掉 is_bad_user 为 True 的用户
users_cleaned = users_deduplicated[users_deduplicated['is_bad_user'] == False].copy()
edu_cleaned = edu[edu['is_bad_user'] == False].copy()
print(f"Shape of the filtered users table: {users_cleaned.shape}")
print(f"Shape of the filtered edu table: {edu_cleaned.shape}\n")


# 1b. 定义并保留我们需要的特征列
user_features = [
    'user_id', 'numconnections', 'f_prob', 'm_prob', 'white_prob', 'black_prob',
    'api_prob', 'hispanic_prob', 'native_prob', 'multiple_prob',
    'highest_degree', 'currentindustry'
]
position_features = [
    'user_id', 'company_cleaned', 'region', 'country', 'state', 'city',
    'mapped_role', 'seniority', 'total_compensation', 'rn',
    'ultimate_parent_company_name', 'startdate', 'enddate'
]
edu_features = [
    'user_id', 'campus_cleaned', 'field_raw', 'degree', 'enddate'
]

users_final = users_cleaned[user_features].copy()
positions_final = positions[position_features].copy()
edu_final = edu_cleaned[edu_features].copy()

print(f"Shape of the filtered users_final table: {users_final.shape}")
print(f"Shape of the filtered positions_final table: {positions_final.shape}")
print(f"Shape of the filtered edu_final table: {edu_final.shape}\n")
print("Feature selection complete.")

Shape of the filtered users table: (4755, 29)
Shape of the filtered edu table: (11398, 25)

Shape of the filtered users_final table: (4755, 12)
Shape of the filtered positions_final table: (39066, 13)
Shape of the filtered edu_final table: (11398, 5)

Feature selection complete.


In [9]:
# --- 步骤 2: 创建新特征 - total_experience_years ---

# 2a. 将 start_date 和 end_date 转换为日期格式
# errors='coerce' 会将无法转换的日期变成 NaT (Not a Time)，避免程序出错
positions_final['startdate'] = pd.to_datetime(positions_final['startdate'], errors='coerce')
positions_final['enddate'] = pd.to_datetime(positions_final['enddate'], errors='coerce')

# 2b. 计算每段职位的时长（天数）
positions_final['duration_days'] = (positions_final['enddate'] - positions_final['startdate']).dt.days

# 2c. 按 user_id 分组，计算总天数，然后转换为年
# 使用 dropna() 确保在计算 sum 之前去掉无效的时长
experience = positions_final.dropna(subset=['duration_days']).groupby('user_id')['duration_days'].sum().reset_index()
experience['total_experience_years'] = experience['duration_days'] / 365.25 # 使用365.25更精确
experience['total_experience_years'] = experience['total_experience_years'].round(2)

# 2d. 将新特征合并回我们的主 users 表
users_final = pd.merge(users_final, experience[['user_id', 'total_experience_years']], on='user_id', how='left')

print("total_experience_years calculated and merged completed")
print(users_final[['user_id', 'total_experience_years']].head(), "\n")

total_experience_years calculated and merged completed
   user_id  total_experience_years
0  1241390                    7.00
1  1249886                    4.75
2  1284328                    6.75
3  1490244                    5.17
4  1511359                    7.49 



In [10]:
# --- 步骤 3: 聚合职位和教育信息 ---
# 我们的目标是每个用户只有一行，所以需要从多条职位/教育记录中选择最相关的一条。

print("Step 3: Aggregating latest position and education info...")

# 3a. Create latest_positions: Ensure it has unique user_ids
# Sort by user_id, then by 'rn' descending to get the latest position first
latest_positions = positions_final.sort_values(by=['user_id', 'rn'], ascending=[True, False])
# Use drop_duplicates to keep only the first record for each user (which is now the latest)
latest_positions = latest_positions.drop_duplicates(subset='user_id', keep='first').copy()
# Drop columns that are no longer needed
latest_positions = latest_positions.drop(columns=['startdate', 'enddate', 'rn', 'duration_days'])


print(f"\n After deduplication, the shape of the `latest_positions` table: {latest_positions.shape}")
print(f"After deduplication, the unique id count in the `latest_positions` table: {latest_positions['user_id'].nunique()}")
print("✅ The latest_positions table shows that there is only one record for each user_id.。\n")

Step 3: Aggregating latest position and education info...

 After deduplication, the shape of the `latest_positions` table: (4655, 10)
After deduplication, the unique id count in the `latest_positions` table: 4655
✅ The latest_positions table shows that there is only one record for each user_id.。



In [11]:
# 3b. Create latest_edu: Ensure it has unique user_ids
edu_final['enddate'] = pd.to_datetime(edu_final['enddate'], errors='coerce')
# Sort by user_id, then by 'enddate' descending to get the most recent education first
latest_edu = edu_final.sort_values(by=['user_id', 'enddate'], ascending=[True, False])
# Use drop_duplicates to keep only the first record for each user
latest_edu = latest_edu.drop_duplicates(subset='user_id', keep='first').copy()
# Drop the helper column
latest_edu = latest_edu.drop(columns=['enddate'])

print(f"\n After deduplication, the shape of the `latest_edu` table: {latest_edu.shape}")
print(f"After deduplication, the unique id count in the `latest_edu` table: {latest_edu['user_id'].nunique()}")
print("✅ The latest_edu table shows that there is only one record for each user_id.\n")


 After deduplication, the shape of the `latest_edu` table: (4519, 4)
After deduplication, the unique id count in the `latest_edu` table: 4519
✅ The latest_edu table shows that there is only one record for each user_id.



In [12]:
print("  - Merging all data....")
# Perform the first LEFT merge
user_feature_table = pd.merge(users_final, latest_positions, on='user_id', how='left')

# Perform the second LEFT merge
user_feature_table = pd.merge(user_feature_table, latest_edu, on='user_id', how='left')

  - Merging all data....


In [13]:
print(f"Filtered users_final table shape: {users_final.shape}\n")
print(f"Filtered positions_final table shape: {positions_final.shape}\n")
print(f"Filtered edu_final table shape: {edu_final.shape}\n")
print(f"user_feature_table table shape: {user_feature_table.shape}\n")

Filtered users_final table shape: (4755, 13)

Filtered positions_final table shape: (39066, 14)

Filtered edu_final table shape: (11398, 5)

user_feature_table table shape: (4755, 25)



In [14]:
# --- Final Verification ---
print("\n--- Verification After Fix ---")
total_rows = len(user_feature_table)
unique_users = user_feature_table['user_id'].nunique()

print(f"Final table total rows: {total_rows}")
print(f"Final table unique user_ids: {unique_users}")


--- Verification After Fix ---
Final table total rows: 4755
Final table unique user_ids: 4755


In [15]:
# --- 步骤 4: 构建最终的用户特征宽表 ---
print("步骤 4: Build the final user feature table...")
print(f"Final table shape: {user_feature_table.shape}")
print("\nFinal table preview:")
from IPython.display import display
display(user_feature_table.head())

print("\n Table information overview:")
user_feature_table.info()

步骤 4: Build the final user feature table...
Final table shape: (4755, 25)

Final table preview:


,user_id,numconnections,f_prob,m_prob,white_prob,black_prob,api_prob,hispanic_prob,native_prob,multiple_prob,...,country,state,city,mapped_role,seniority,total_compensation,ultimate_parent_company_name,campus_cleaned,field_raw,degree
0,1241390,500,0.004899,0.995101,0.993,0.001,0.001,0.003,0.001,0.001,...,United States,New York,New York City,media relations,5.0,196593.703125,SBI Holdings Corp.,texas christian university,Strategic Communication,Bachelor
1,1249886,439,0.004071,0.995929,0.980,0.018,0.001,0.001,0.000,0.000,...,United States,New York,New York City,producer,5.0,412561.937500,Adolescent,parsons school of design the new school,Photography,Bachelor
2,1284328,384,0.576329,0.423671,0.030,0.538,0.354,0.002,0.009,0.067,...,United States,New York,New York City,teaching,4.0,129657.320312,"The NewYork-Presbyterian Healthcare System, Inc.",banaras hindu university,Biotechnology,Doctor
3,1490244,217,0.405578,0.594422,0.008,0.001,0.980,0.000,0.000,0.011,...,United States,New York,New York City,research scientist,2.0,165176.078125,"Meta Platforms, Inc.",iowa state university,Electrical and Electronics Engineering,Doctor
4,1511359,274,0.998887,0.001113,0.988,0.010,0.001,0.001,0.000,0.000,...,United States,New York,New York City,research,1.0,108993.890625,The Trustees of Columbia University in The Cit...,california polytechnic state university san lu...,Mathematics,Bachelor



 Table information overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4755 entries, 0 to 4754
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   user_id                       4755 non-null   int64  
 1   numconnections                4755 non-null   int64  
 2   f_prob                        4755 non-null   float64
 3   m_prob                        4755 non-null   float64
 4   white_prob                    4755 non-null   float64
 5   black_prob                    4755 non-null   float64
 6   api_prob                      4755 non-null   float64
 7   hispanic_prob                 4755 non-null   float64
 8   native_prob                   4755 non-null   float64
 9   multiple_prob                 4755 non-null   float64
 10  highest_degree                4755 non-null   object 
 11  currentindustry               4389 non-null   object 
 12  total_experience_years        43

In [16]:
missing_values = user_feature_table.isnull().sum()
print(missing_values)

user_id                            0
numconnections                     0
f_prob                             0
m_prob                             0
white_prob                         0
black_prob                         0
api_prob                           0
hispanic_prob                      0
native_prob                        0
multiple_prob                      0
highest_degree                     0
currentindustry                  366
total_experience_years           420
company_cleaned                  104
region                           102
country                          102
state                            102
city                             102
mapped_role                      102
seniority                        102
total_compensation               102
ultimate_parent_company_name     576
campus_cleaned                   236
field_raw                       1034
degree                           236
dtype: int64


In [17]:
# --- 步骤 5: 将最终的特征表保存为CSV文件 ---
print("Saving the user_feature_table as a CSV file....")

output_filename = 'user_features.csv'

user_feature_table.to_csv(output_filename, index=False)

print(f"\n✅ The file '{output_filename}' has been successfully saved to the current directory in Colab.")
print("You can now see it in the 'Files' panel on the left, and right-click to download.")

Saving the user_feature_table as a CSV file....

✅ The file 'user_features.csv' has been successfully saved to the current directory in Colab.
You can now see it in the 'Files' panel on the left, and right-click to download.
